# Evaluation of 1 Model With Physgen
-> Phys Anything

In [ ]:
model_name = "phys_any_1"
physgen_variation = "sound_reflection"    # sound_baseline, sound_reflection, sound_diffraction, sound_combined
encoder = "vitl"    # vits, vitb, vitl, vitg

### Env Setup

Follow the README  env installation steps and set 'phy_any' as the active python env.

### Imports

In [ ]:
import os
import re
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cv2

In [ ]:
!python --version

### Helper

In [ ]:
def imshow(img, title=None, image_width=10, axis=False,
           color_space="RGB", cmap=None, cols=1, save_to=None,
           hspace=0.2, wspace=0.2,
           use_original_sytle=False, invert=False):
    """
    Visualizes one or multiple images.

    Image will be reshaped: [batch_size/images, width, height, channels]

    ---
    Parameters:
    - img : np.ndarray
        Images/Images with [width, height, channels] or for multiple: [batch_size/images, width, height, channels].
    - title : str, optional (default=None)
        Title of the whole plot.
    - image_width : int, optional (default=5)
        Width of one image in the plot.
    - axis : bool, optional (default=False)
        Whether to print the axis of the images or not.
    - color_space : str, optional (default="RGB")
        The colorspace of the image: RGB, BGR, gray, HSV.
    - cmap : str, optional (default=None)
        Which cmap to use. Check all cmaps here out: https://matplotlib.org/stable/users/explain/colors/colormaps.html
    - cols : int, optional (default=1)
        Amount of columns in the plot.
    - save_to : str, optional (default=None)
        Path where to save the result image.
    - hspace : float, optional (default=0.01)
        Horizontal space between the images.
    - wspace : float, optional (default=0.01)
        Vertical space between the images.
    - use_original_sytle : bool, optonial (default=False)
        Whether the plot should use the current active matplotlib style or choosing a own one. 
    - invert : bool, optional (default=False)
        Whether to invert the images or not.
    """
    original_style = plt.rcParams.copy()

    img_shape = img.shape
    print(f"Got images with shape: {img_shape}")

    # tranform the image to the right form
    if len(img_shape) == 2:
        img = np.reshape(img, shape=(1, img.shape[0], img.shape[1], 1))
    elif len(img_shape) == 3:
        # check if multiple gray images or multiple images with channel
        # if img.shape[2] < img.shape[0] and img.shape[1] == img.shape[2]:
        img = np.reshape(img, shape=(1, img.shape[0], img.shape[1], img.shape[2]))
        # else:
        #     # there could be cases where this is wrong
        #     img = np.reshape(img, shape=(img.shape[0], img.shape[1], img.shape[2], 1))
    elif len(img_shape) != 4:
        raise ValueError(f"Image(s) have wrong shape! Founded shape: {img.shape}.")

    print(f"Transformed shape to: {img_shape}")

    # invert images
    if invert:
        print("Invert images...")
        max_value = 2**(img.dtype.itemsize * 8) -1
        scaling_func = lambda x: max_value - x
        img = np.apply_along_axis(scaling_func, axis=0, arr=img)

    # Set visualization settings
    # aspect_ratio_width = img.shape[1] / img.shape[2]
    aspect_ratio = img.shape[2] / img.shape[1]

    n_images = img.shape[0]
    rows = n_images//cols + int(n_images % cols > 0)

    width = int(image_width * cols)
    height = int(image_width * rows * aspect_ratio)

    # set plt style
    if not use_original_sytle:
        plt_style = 'seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else np.random.choice(plt.style.available)
        plt.style.use(plt_style)
        print(f"Using '{plt_style}' plotting style.")

    # plotting
    print(f"Making you a beautiful plot...")
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(width, height))
    try:
        ax = ax.ravel()
    except AttributeError:
        ax = [ax]
    fig.subplots_adjust(hspace=hspace, wspace=wspace)
    if type(title) == str:
        fig.suptitle(title, fontsize=128, y=0.95)

    for idx in range(len(ax)):
        cur_ax = ax[idx]

        if idx >= len(img):
            cur_ax.axis("off")
            continue

        cur_img = img[idx]

        if color_space.lower() == "bgr":
            cur_img = cv2.cvtColor(cur_img, cv2.COLOR_BGR2RGB)
            used_cmap = None
        elif color_space.lower() == "rgb":
            cur_img = cur_img
            used_cmap = None
        elif color_space.lower() == "hsv":
            cur_img = cv2.cvtColor(cur_img, cv2.COLOR_HSV2RGB)
            used_cmap = None
        elif color_space.lower() in ["gray", "grey", "g"]:
            if len(cur_img.shape) == 3 and cur_img.shape[2]:
                cur_img = cv2.cvtColor(cur_img, cv2.COLOR_RGB2GRAY)
            else:
                cur_img = cur_img
            print(cur_img.shape)
            used_cmap = "gray"

        if cmap:
            used_cmap = cmap

        if type(title) in [list, tuple]:
            cur_ax.set_title(title[idx], fontsize=64)
        if axis == False:
            cur_ax.axis("off")

        cur_ax.imshow(cur_img, cmap=used_cmap)

    if save_to:
        os.makedirs(os.path.split(save_to)[0], exist_ok=True)
        fig.savefig(save_to, dpi=300)

    plt.show()

    if not use_original_sytle:
        # reset to original plt style
        plt.rcParams.update(original_style)

def show_images(image_paths:list, title=None, image_width=5, axis=False,
                color_space="gray", cmap=None, 
                cols=2, save_to=None,
                hspace=0.01, wspace=0.01,
                use_original_sytle=False, invert=False):
    """
    Visulalizes/shows one or multiple images.

    ---
    Parameters:
    - image_paths : List[str]
        List of paths to the images which should get visualized.
    - title : str, optional (default=None)
        Title of the whole plot.
    - image_width : int, optional (default=5)
        Width of one image in the plot.
    - axis : bool, optional (default=False)
        Whether to print the axis of the images or not.
    - color_space : str, optional (default="RGB")
        The colorspace of the image: RGB, BGR, gray, HSV.
    - cmap : str, optional (default=None)
        Which cmap to use. Check all cmaps here out: https://matplotlib.org/stable/users/explain/colors/colormaps.html
    - cols : int, optional (default=1)
        Amount of columns in the plot.
    - save_to : str, optional (default=None)
        Path where to save the result image.
    - hspace : float, optional (default=0.01)
        Horizontal space between the images.
    - wspace : float, optional (default=0.01)
        Vertical space between the images.
    - use_original_sytle : bool, optonial (default=False)
        Whether the plot should use the current active matplotlib style or choosing a own one. 
    - invert : bool, optional (default=False)
        Whether to invert the images or not.
    """
    if color_space.lower() == "rgb":
        images = np.array([cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB) for img in image_paths])
    elif color_space.lower() == "hsv":
        images = np.array([cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2HSV) for img in image_paths])
    else:
        images = np.array([cv2.imread(img) for img in image_paths])
    imshow(images, title=title, image_width=image_width, axis=axis,
           color_space=color_space, cmap=cmap, cols=cols, save_to=save_to,
           hspace=hspace, wspace=wspace,
           use_original_sytle=use_original_sytle, invert=invert)
    return images

### Run Inference

In [ ]:
command = (
  f"python inference.py "
  f"--variation {variation} "
  f"--model_name {model_name} "
  f"--encoder {encoder} "
  # f"--save_only_result"
)

# Finally run it
!{command}

### Calc Eval metrics

In [ ]:
!python eval_metrics.py \
    --data_dir ./eval/{model_name}/real \
    --pred_dir ./eval/{model_name}/pred \
    --osm_dir ./eval/{model_name}/osm \
    --output ./eval_results/evaluation_{model_name}.csv

### Show Results

In [ ]:
mae_model_name = f'MAE_{model_name}'
los_mae_model_name = f'LoS_MAE_{model_name}'
nlos_mae_model_name = f'NLoS_MAE_{model_name}'
mape_model_name = f'MAPE_{model_name}'
los_wmape_model_name = f'LoS_wMAPE_{model_name}'
nlos_wmape_model_name = f'NLoS_wMAPE_{model_name}'

In [ ]:
df_1 = pd.read_csv(f"./eval_results/evaluation_{model_name}.csv")
# df_1 = df_1.drop(columns=["LoS_MAE", "NLoS_MAE", "LoS_wMAPE", "NLoS_wMAPE"])
df_1 = df_1.rename(columns={'MAE': mae_model_name, 
                            'LoS_MAE': los_mae_model_name,
                            'NLoS_MAE': nlos_mae_model_name,
                            'MAPE':mape_model_name,
                            'LoS_wMAPE': los_wmape_model_name,
                            'NLoS_wMAPE': nlos_wmape_model_name
                            }
                   )
# extract sample_ids
sample_id_series = df_1["sample_id"].str.extract(r'^(\d+)_')[0]
if sample_id_series.isna().sum() > 1:
    sample_id_series = df_1["sample_id"].str.extract(r'(\d+)')[0]
if sample_id_series.isna().sum() > 1:
    raise ValueError(f"Found {sample_id_series.isna().sum()} Nans")
df_1["sample_id"] = sample_id_series
print("Nan found in sample ids:", df_1["sample_id"].isna().sum())
df_1 = df_1.dropna(subset=["sample_id"])
df_1["sample_id"] = df_1["sample_id"].astype(int)
merged_df = df_1
df_1

In [ ]:
plt_style = 'seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else np.random.choice(plt.style.available)
plt.style.use(plt_style)
print(f"Using '{plt_style}' plotting style.")

values = [
    merged_df[mae_model_name],
    merged_df[mape_model_name],
    merged_df[los_mae_model_name],
    merged_df[nlos_mae_model_name],
    merged_df[los_wmape_model_name],
    merged_df[nlos_wmape_model_name]
]

labels = [
    "MAE",
    "MAPE",
    "LoS MAE",
    "NLoS MAE",
    "LoS wMAPE",
    "NLoS wMAPE"
]

fig, ax = plt.subplots(figsize=(12, 6))
ax.boxplot(values, notch=False)
ax.set_xticks(range(1, len(labels) + 1))
ax.set_xticklabels(labels, rotation=15)
ax.set_title("Error Metrics Distribution")



print(f"\nMAE: {merged_df[mae_model_name].mean():>0.2f}")
print(f"\nMAPE: {merged_df[mape_model_name].mean():>0.2f}")
print(f"\nLoS MAE: {merged_df[los_mae_model_name].mean():>0.2f}")
print(f"\nNLoS MAE: {merged_df[nlos_mae_model_name].mean():>0.2f}")
print(f"\nLoS wMAPE: {merged_df[los_wmape_model_name].mean():>0.2f}")
print(f"\nNLoS wMAPE: {merged_df[nlos_wmape_model_name].mean():>0.2f}")

Example Image

In [ ]:
def get_same_pred_real_samples(pred_path:str, real_path:str, n_samples:int, ids=None):
    if not ids:
        # choose n random samples
        samples = random.sample(os.listdir(pred_path), n_samples)
        result_samples = [os.path.join(pred_path, cur_image) for cur_image in samples]

        # get the used id's
        ids = []
        for cur_image in samples:
            cur_id = re.findall(r'\d+', string=cur_image)
            if len(cur_id) > 1:
                raise ValueError("Too many ids found!")
            cur_id = cur_id[0]
            if len(cur_id) <= 0:
                raise ValueError(f"One image has no ID: {cur_image}")
            ids += [cur_id]
    else:
        # get pred image
        pred_image_samples = []
        for target_id in ids:
            found = False
            for cur_image in os.listdir(real_path):
                cur_id = re.findall('\d+', string=cur_image)
                if len(cur_id) > 1:
                    raise ValueError(f"Too many ids found in {cur_image}!")
                elif len(cur_id) <= 0:
                    continue
                    # raise ValueError(f"No id found in {cur_image}!")
                cur_id = cur_id[0]
                if cur_id == target_id:
                    pred_image_samples += [cur_image]
                    found = True
                    break

            if not found:
                raise ValueError(f"Does not found pred image with id: {target_id}")
        result_samples = [os.path.join(pred_path, cur_image) for cur_image in pred_image_samples]

    # get real image
    real_image_samples = []
    for target_id in ids:
        found = False
        for cur_image in os.listdir(real_path):
            cur_id = re.findall('\d+', string=cur_image)
            if len(cur_id) > 1:
                raise ValueError(f"Too many ids found in {cur_image}!")
            elif len(cur_id) <= 0:
                continue
                # raise ValueError(f"No id found in {cur_image}!")
            cur_id = cur_id[0]
            if cur_id == target_id:
                real_image_samples += [cur_image]
                found = True
                break

        if not found:
            raise ValueError(f"Does not found real image with id: {target_id}")

    result_samples += [os.path.join(real_path, cur_image) for cur_image in real_image_samples]

    return result_samples, ids

In [ ]:
from matplotlib.colors import ListedColormap

def get_unique_hsv_cmap():
    unique_hsv_map = plt.get_cmap("hsv")(np.linspace(0, 1, 256))    # np.arange(0, 256)
    hsv_map = plt.get_cmap("hsv")
    for cur_idx in range(256):
        r, g, b, a = hsv_map(cur_idx)
        if r > 0.99 and g < (170/255):
            gray_value = cur_idx*8 / 255.0
            unique_hsv_map[cur_idx] = (gray_value, gray_value, gray_value, 1.0)
        else:
            break
    unique_hsv = ListedColormap(unique_hsv_map)
    plt.colormaps.register(name="unique_hsv", cmap=unique_hsv, force=True)
    return unique_hsv

# for i in range(256):
#     print([int(cur_color*255) for cur_color in get_cmap('hsv')(i)])

get_unique_hsv_cmap()
plt.get_cmap("unique_hsv")

In [ ]:
def plot(ax, path, title="", invert=True, sub_image=None, cmap="unique_hsv", scale=False, logscale=False, plot=True):
    img = np.array(cv2.imread(path, cv2.IMREAD_GRAYSCALE))

    if sub_image:
        img_2 = np.array(cv2.imread(sub_image, cv2.IMREAD_GRAYSCALE))
        # img = cv2.subtract(img, img_2)
        img = img - img_2
        img[img < 0] = img[img < 0] * -1
        # img = np.abs(img - img_2)
    
    # # scaling
    # if scale:
    #     mask = (img > 0) & (img < 160)
    #     img[mask] = np.clip(img[mask] + 40, 0, 255) # cv2.add(img[mask], 30)

    # invert
    if invert:
        max_value = 255 # 2**(img.dtype.itemsize * 8) -1
        scaling_func = lambda x: max_value - x
        img = np.apply_along_axis(scaling_func, axis=0, arr=img)

    # # Apply logarithmic scaling to enhance contrast in dim regions
    # if logscale:
    #     img = np.log1p(img)
    #     # img /= img.max()  # normalize to [0,1]

    if plot:
        ax.axis("off")
        color_ax = ax.imshow(img, cmap=cmap)
        ax.set_title(title)
    
    return img

In [ ]:
n_samples = 5

example_images_model, ids = get_same_pred_real_samples(f"../../data/eval/{model_name}/pred",
                                                         f"../../data/eval/{model_name}/real",
                                                         n_samples)

pred_model = example_images_model[:n_samples]
real = example_images_model[n_samples:] 

fig, ax = plt.subplots(nrows=n_samples, ncols=2, figsize=(2*4, n_samples*6))
# ax = ax.ravel()

ax_idx = 0

for idx, cur_path in enumerate(pred_model):
    plot(ax[idx][0], path=cur_path, title=f"{model_name}")

for idx, cur_path in enumerate(real):
    plot(ax[idx][1], path=cur_path, title=f"ground truth")

plt.subplots_adjust(hspace=0.5)

plt.show();

In [ ]:
fig, ax = plt.subplots(nrows=n_samples, ncols=2, figsize=(2*4, n_samples*6))

ax_idx = 0

for idx, cur_path in enumerate(pred_model):
    plot(ax[idx][0], path=cur_path, title=f"{model_name}", cmap="plasma")

for idx, cur_path in enumerate(real):
    plot(ax[idx][1], path=cur_path, title=f"ground truth", cmap="plasma")

plt.subplots_adjust(hspace=0.5)

plt.show();

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))

# Create a mappable object (for the colorbar)
norm = plt.Normalize(vmin=0, vmax=255)
sm = plt.cm.ScalarMappable(cmap="unique_hsv", norm=norm)
sm.set_array([])  # Required to set the color range without linking it to any image

fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.05, pad=0.4)

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=n_samples, ncols=1, figsize=(1*4, n_samples*6))

for idx, cur_path in enumerate(pred_model):
    plot(ax[idx], path=cur_path, title=f"{model_name}", sub_image=real[idx], invert=False, scale=True, cmap="plasma")

plt.subplots_adjust(hspace=0.5)
plt.show();

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))

# Create a mappable object (for the colorbar)
norm = plt.Normalize(vmin=0, vmax=255)
sm = plt.cm.ScalarMappable(cmap="plasma", norm=norm)
sm.set_array([])  # Required to set the color range without linking it to any image

fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.05, pad=0.4)

plt.show()

Inspect some single images in more detail here:

In [ ]:
from skimage.measure import block_reduce

def plot_image_with_values(img, block_size=8):
    # Compute mean over non-overlapping blocks
    mean_img = block_reduce(img, block_size=(block_size, block_size), func=np.mean)
    max_value = mean_img.max()

    # Plot the mean image
    plt.imshow(mean_img, cmap='gray', interpolation='nearest')
    plt.colorbar(label='Mean Value')

    # Annotate each block with the mean
    for i in range(mean_img.shape[0]):
        for j in range(mean_img.shape[1]):
            val = mean_img[i, j]
            color = 'white' if val < max_value/1.5 else 'black'
            # color = int(255 - val)
            plt.text(j, i, f'{val:.1f}', ha='center', va='center',
                     color=color, fontsize=6)

    plt.title(f'Mean Values over {block_size}x{block_size} Blocks')
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.show()

In [ ]:
img_path = pred_model[0]
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE).astype(float)

plot_image_with_values(img, block_size=16)

In [ ]:
img = plot(None, path=pred_model[0], title=f"{model_name}", sub_image=real[idx], invert=False, scale=True, plot=False)
plot_image_with_values(img, block_size=16)

### Old Code /Visualization

In [ ]:
path = f"../../data/eval/{model_name}/pred"
example_images = [os.path.join(path, cur_img) for cur_img in os.listdir(path)]
print(f"Examples from {model_name}")
show_images(image_paths=example_images[:10], image_width=4, cols=5, cmap="inferno", color_space="grey", invert=True);

In [ ]:
from matplotlib.ticker import MultipleLocator

example_images_model, ids = get_same_pred_real_samples(f"../../data/eval/{model_name}/pred",
                                                         f"../../data/eval/{model_name}/real",
                                                         n_samples)

# Plot the histogram
for cur_image in example_images_model:
    img = np.array(cv2.imread(cur_image, cv2.IMREAD_GRAYSCALE))
    plt.hist(img.flatten(), bins=256, range=(190, 210), density=True)
    plt.xlabel("Pixel Values")
    plt.ylabel("Frequency")

    # Set every x-tick (e.g., every 1 value from 190 to 255)
    plt.gca().xaxis.set_major_locator(MultipleLocator(1))
    plt.xticks(rotation=45)

    plt.show()

In [ ]:
for cur_image in example_images_model:
    img = np.array(cv2.imread(cur_image, cv2.IMREAD_GRAYSCALE))
    plt.hist(img.flatten(), bins=256, range=(0, 256), density=True)
    plt.xlabel("Pixel Values")
    plt.ylabel("Frequency")
    plt.yscale('log')
    plt.show()

### Check Processed Images

for quantization error

In [ ]:
processed_example_images_model = []
for cur_image in example_images_model:
    processed_example_images_model += [plot(None, path=cur_path, title=f"ground truth", plot=False)]

In [ ]:
for cur_image in processed_example_images_model:
    img = cur_image
    plt.hist(img.flatten(), bins=256, range=(0, 256), density=True)
    plt.xlabel("Pixel Values")
    plt.ylabel("Frequency")
    plt.yscale("log")
    plt.show()

In [ ]:
for cur_image in processed_example_images_model:
    img = cur_image
    plt.hist(img.flatten(), bins=256, range=(256-210, 256-190), density=True)
    plt.xlabel("Pixel Values")
    plt.ylabel("Frequency")

    # Set every x-tick (e.g., every 1 value from 190 to 255)
    plt.gca().xaxis.set_major_locator(MultipleLocator(1))
    plt.xticks(rotation=45)

    plt.show()